In [2]:
import numpy as np
import pandas as pd
import torch
torch.cuda.is_available()

True

In [1]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
from torchvision import models

In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

DATA_DIR = "../../data/processed/ready"

def get_dataloaders(batch_size=32, val_split=0.2):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

    dataset = datasets.ImageFolder(DATA_DIR, transform=transform)

    val_size = int(len(dataset) * val_split)
    train_size = len(dataset) - val_size

    train_ds, val_ds = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, len(dataset.classes)
import torch.nn as nn
from torchvision import models

def get_model(num_classes):
    model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

    for param in model.parameters():
        param.requires_grad = False

    model.fc = nn.Linear(model.fc.in_features, num_classes)

    return model


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
EPOCHS = 3
LR = 1e-3
BATCH_SIZE = 32

def evaluate(model, loader, criterion):
    model.eval()
    total, correct, loss_sum = 0, 0, 0

    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            out = model(x)
            loss = criterion(out, y)

            loss_sum += loss.item()
            preds = out.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)

    return loss_sum / len(loader), correct / total

def main():
    train_loader, val_loader, num_classes = get_dataloaders(BATCH_SIZE)
    model = get_model(num_classes).to(DEVICE)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.fc.parameters(), lr=LR)

    for epoch in range(EPOCHS):
        model.train()
        for x,y in tqdm(train_loader):
            x, y = x.to(DEVICE), y.to(DEVICE)

            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()

        val_loss, val_acc = evaluate(model, val_loader, criterion)
        print(f"Epoch {epoch+1}: val_loss = {val_loss:.4f}, val_accuracy = {val_acc:.4f}")
    os.makedirs("../../models", exist_ok=True)
    torch.save(model.state_dict(), "../../models/resnet18.pt")

In [32]:
main()

100% 41/41 [00:01<00:00, 25.15it/s]


Epoch 1: val_loss = 1.5483, val_accuracy = 0.4503


100% 41/41 [00:01<00:00, 27.08it/s]


Epoch 2: val_loss = 1.2427, val_accuracy = 0.6149


100% 41/41 [00:01<00:00, 30.18it/s]


Epoch 3: val_loss = 1.1578, val_accuracy = 0.6366
